In [21]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from scikeras.wrappers import KerasClassifier

In [22]:
input_data = pd.read_csv("train.csv", index_col='PassengerId')

def set_age(name):
    if(name.find('Mr.') != -1): #male - married or not
        return 30
    elif(name.find('Mrs.') != -1): #female - married
        return 45
    elif(name.find('Ms.') != -1): #female - married or not 
        return 30
    elif(name.find('Miss.') != -1): #female - not married 
        return 20
    else:
        return 45

def clean_dataset(dataset):
    popular_port = dataset['Embarked'].dropna().mode()[0] 
    print("This port is most popular and replace all 'NA' to " + popular_port)
    dataset['Embarked'] = dataset['Embarked'].fillna(popular_port) 
    dataset['Companions'] = dataset['SibSp'] + dataset['Parch'] #number of people travelling with
    dataset['Female'] = dataset['Sex'].map({'female': 1, 'male': 0})
    dataset['Male'] = dataset['Sex'].map({'female': 0, 'male': 1})
    embarked_one_hot = pd.get_dummies(dataset['Embarked'], prefix='Embarked') #Embarks as one-hot
    dataset = dataset.join(embarked_one_hot) 
    dataset['Age'] = dataset.apply(
                            lambda row: set_age(row['Name']) if np.isnan(row['Age']) else row['Age'], axis=1)
    
    dataset = dataset.drop(['Cabin', 'Ticket', 'Name', 'SibSp', 'Parch', 'Embarked', 'Sex'], axis=1) #drop useless columns
    return(dataset)
    
    
train_data = clean_dataset(input_data)
print("Prepared data:")
train_data

This port is most popular and replace all 'NA' to S
Prepared data:


,Survived,Pclass,Age,Fare,Companions,Female,Male,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,,,,,,,,
1,0,3,22.0,7.2500,1,0,1,0,0,1
2,1,1,38.0,71.2833,1,1,0,1,0,0
3,1,3,26.0,7.9250,0,1,0,0,0,1
4,1,1,35.0,53.1000,1,1,0,0,0,1
5,0,3,35.0,8.0500,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...
887,0,2,27.0,13.0000,0,0,1,0,0,1
888,1,1,19.0,30.0000,0,1,0,0,0,1
889,0,3,20.0,23.4500,3,1,0,0,0,1


In [23]:
 # X and Y ('survived;) axis
 X = train_data.drop(['Survived'], axis=1).values.astype(float)

 Y = train_data['Survived'].values

 def create_model(optimizer='adam', init='uniform'):
     #create model
     model = keras.Sequential()
     model.add( keras.layers.Dense(16, input_dim=X.shape[1], kernel_initializer=init, activation='relu') )
     model.add( keras.layers.Dense(8, kernel_initializer=init, activation='relu') )
     model.add( keras.layers.Dense(4, kernel_initializer=init, activation='relu') )
     model.add( keras.layers.Dense(1, kernel_initializer=init, activation='sigmoid') )

     #compile model
     model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
     print("Model created")
     return model

In [24]:
epochs = 200
batch_size = 5
init = 'glorot_uniform'
optimizer = 'rmsprop'

#create classfier
model_predicitons = KerasClassifier(
    model=create_model, optimizer=optimizer, init=init, epochs=epochs,
    batch_size=batch_size)

model_predicitons.fit(X,Y)

Model created
Epoch 1/200
179/179 [==============================] - 0s 880us/step - loss: 3.3383 - accuracy: 0.4097
Epoch 2/200
179/179 [==============================] - 0s 898us/step - loss: 0.6811 - accuracy: 0.6117
Epoch 3/200
179/179 [==============================] - 0s 951us/step - loss: 0.6717 - accuracy: 0.6308
Epoch 4/200
179/179 [==============================] - 0s 1ms/step - loss: 0.6661 - accuracy: 0.6409
Epoch 5/200
179/179 [==============================] - 0s 1ms/step - loss: 0.6671 - accuracy: 0.6308
Epoch 6/200
179/179 [==============================] - 0s 1ms/step - loss: 0.6577 - accuracy: 0.6386
Epoch 7/200
179/179 [==============================] - 0s 832us/step - loss: 0.6580 - accuracy: 0.6397
Epoch 8/200
179/179 [==============================] - 0s 830us/step - loss: 0.6526 - accuracy: 0.6498
Epoch 9/200
179/179 [==============================] - 0s 820us/step - loss: 0.6404 - accuracy: 0.6678
Epoch 10/200
179/179 [==============================] - 0s 839us/

179/179 [==============================] - 0s 1ms/step - loss: 0.4062 - accuracy: 0.8204
Epoch 80/200
179/179 [==============================] - 0s 936us/step - loss: 0.4143 - accuracy: 0.8182
Epoch 81/200
179/179 [==============================] - 0s 807us/step - loss: 0.4071 - accuracy: 0.8193
Epoch 82/200
179/179 [==============================] - 0s 821us/step - loss: 0.4080 - accuracy: 0.8171
Epoch 83/200
179/179 [==============================] - 0s 793us/step - loss: 0.4043 - accuracy: 0.8249
Epoch 84/200
179/179 [==============================] - 0s 821us/step - loss: 0.4014 - accuracy: 0.8171
Epoch 85/200
179/179 [==============================] - 0s 810us/step - loss: 0.4039 - accuracy: 0.8159
Epoch 86/200
179/179 [==============================] - 0s 794us/step - loss: 0.4062 - accuracy: 0.8238
Epoch 87/200
179/179 [==============================] - 0s 787us/step - loss: 0.3993 - accuracy: 0.8227
Epoch 88/200
179/179 [==============================] - 0s 789us/step - loss: 0

179/179 [==============================] - 0s 882us/step - loss: 0.3749 - accuracy: 0.8350
Epoch 158/200
179/179 [==============================] - 0s 810us/step - loss: 0.3810 - accuracy: 0.8283
Epoch 159/200
179/179 [==============================] - 0s 811us/step - loss: 0.3804 - accuracy: 0.8328
Epoch 160/200
179/179 [==============================] - 0s 829us/step - loss: 0.3762 - accuracy: 0.8350
Epoch 161/200
179/179 [==============================] - 0s 819us/step - loss: 0.3829 - accuracy: 0.8283
Epoch 162/200
179/179 [==============================] - 0s 803us/step - loss: 0.3772 - accuracy: 0.8395
Epoch 163/200
179/179 [==============================] - 0s 775us/step - loss: 0.3751 - accuracy: 0.8316
Epoch 164/200
179/179 [==============================] - 0s 906us/step - loss: 0.3738 - accuracy: 0.8272
Epoch 165/200
179/179 [==============================] - 0s 874us/step - loss: 0.3771 - accuracy: 0.8272
Epoch 166/200
179/179 [==============================] - 0s 838us/ste

KerasClassifier(
	model=<function create_model at 0x135d911f0>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=5
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=200
	init=glorot_uniform
	class_weight=None
)

In [25]:
#pred?
d = {'Pclass': [3], 'Female': [1],'Male': [0], 'Age': [22.0], 'Fare': [78.222], 
            'Companions': [2], 'Embarked_C': [1], 'Embarked_Q': [0], 'Embarked_S':[0]}
df = pd.DataFrame(data=d)

X_data = df.values.astype(float)

prediction = model_predicitons.predict(X_data)

print(prediction)

1/1 [==============================] - 0s 48ms/step
[1]
